In [1]:
import numpy as np
import pandas as pd

import seaborn as sns
import scipy.stats as stats
import statsmodels.api as sm

from matplotlib import pyplot as plt

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, mean_squared_error, r2_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from utils.data_processing import *
from utils.graph_processing import *
from models.logistic_regression import LogisticRegression

We are first loading all possible games: games played before 2011, that were winnable, where there was no timeout. We remove timeouted games as we do not know the reason why the player gave up and it may not be linked with the Wikispeedia game.

In [2]:
games = load_preprocessed_games()
games.head()

Loaded 51318 finished paths in df of shape (51318, 7)
Loaded 24875 unfinished paths in df of shape (24875, 8)
After filtering all paths after 2011-02-07 05:02:15
we kept 23245 paths out of 51318 finished paths
There are 24875 unfinished paths
Loaded 4604 articles in df of shape (4604, 1)
Pruning invalid games. Initially we have 48120 games
Pruned invalid games. Now we have 48092 valid games
After removing timeouted games, there are 38775 games left


,difficulty_rating,duration,finished,hashIP,num_backward,path,path_length,source,target,timestamp,type_end
0,NaN,166,True,6a3701d319fc3754,0,"[14th_century, 15th_century, 16th_century, Pac...",9,14th_century,African_slave_trade,2011-02-15 03:26:49,NaN
1,3.0,88,True,3824310e536af032,0,"[14th_century, Europe, Africa, Atlantic_slave_...",5,14th_century,African_slave_trade,2012-08-12 06:36:52,NaN
2,NaN,138,True,415612e93584d30e,0,"[14th_century, Niger, Nigeria, British_Empire,...",8,14th_century,African_slave_trade,2012-10-03 21:10:40,NaN
3,3.0,175,True,015245d773376aab,0,"[14th_century, Italy, Roman_Catholic_Church, H...",7,14th_century,John_F._Kennedy,2013-04-23 15:27:08,NaN
4,NaN,110,True,5295bca242be81fe,0,"[14th_century, Europe, North_America, United_S...",6,14th_century,John_F._Kennedy,2013-07-03 22:26:54,NaN


Let's see how long games are.

In [3]:
games[(games['path_length'] == 1)].shape

(3204, 11)

In [4]:
print(games[(games['path_length'] == 1) & (games['finished'] == True)].shape)
print(games[(games['path_length'] == 1) & (games['finished'] == True)])

(9, 11)
       difficulty_rating  duration  finished            hashIP  num_backward  \
3908                 NaN         0      True  43f864c75b2571b5             0   
5405                 NaN         0      True  43f864c75b2571b5             0   
11137                NaN         0      True  43f864c75b2571b5             0   
12391                NaN         0      True  43f864c75b2571b5             0   
14685                NaN         0      True  43f864c75b2571b5             0   
15737                NaN         0      True  43f864c75b2571b5             0   
15911                NaN         0      True  43f864c75b2571b5             0   
17716                NaN         0      True  43f864c75b2571b5             0   
22649                NaN         0      True  43f864c75b2571b5             0   

                         path  path_length                source  \
3908                [Lesotho]            1               Lesotho   
5405                   [Coal]            1             

So there are 9 games where the source and target article are the same, so the player automatically won. These games are not useful when analysing games so we will remove them. We still keep the games of path length 1 since the nature of the first article may impact if the player gives up.

In [5]:
condition = (games['path_length'] == 1) & (games['finished'] == True)
games = games[~condition]
games.shape

(38766, 11)

In [6]:
games[(games['path_length'] == 2)].shape

(2233, 11)

In [7]:
games[(games['path_length'] == 3)].shape

(3354, 11)

In [8]:
games[(games['path_length'] > 4)].shape

(24641, 11)

Most of the games have a path length bigger than 4 (at least 3 clicks before the final click), so we will first try to analyse these.

In [9]:
games_longer_than_4 = games.copy()
games_longer_than_4 = games_longer_than_4[(games_longer_than_4['path_length'] > 4)]

The features we will take into account will be the inherent difficulty of the game and informations from the first 3 clicks. The information from the first 3 clicks will be: the shortest path length from the article to the goal, the fame of the article, if the click was a backward click or not, the pagerank of the article, the duration (approximated), the probabilities of the link used being known to people, the pagerank of the target. The target variable is whether the games was won or lost. We do not use information from the first article as it is contained in the inherent difficulty.

In [10]:
games_longer_than_4.head()

,difficulty_rating,duration,finished,hashIP,num_backward,path,path_length,source,target,timestamp,type_end
0,NaN,166,True,6a3701d319fc3754,0,"[14th_century, 15th_century, 16th_century, Pac...",9,14th_century,African_slave_trade,2011-02-15 03:26:49,NaN
1,3.0,88,True,3824310e536af032,0,"[14th_century, Europe, Africa, Atlantic_slave_...",5,14th_century,African_slave_trade,2012-08-12 06:36:52,NaN
2,NaN,138,True,415612e93584d30e,0,"[14th_century, Niger, Nigeria, British_Empire,...",8,14th_century,African_slave_trade,2012-10-03 21:10:40,NaN
3,3.0,175,True,015245d773376aab,0,"[14th_century, Italy, Roman_Catholic_Church, H...",7,14th_century,John_F._Kennedy,2013-04-23 15:27:08,NaN
4,NaN,110,True,5295bca242be81fe,0,"[14th_century, Europe, North_America, United_S...",6,14th_century,John_F._Kennedy,2013-07-03 22:26:54,NaN


In [11]:
games_longer_than_4['duration'] = (3/(games_longer_than_4['path_length']-1))* games_longer_than_4['duration']

In [12]:
games_longer_than_4.drop(columns = ["difficulty_rating", 'hashIP', 'num_backward', 'path_length', 'type_end', 'timestamp'], inplace = True)

In [13]:
games_longer_than_4.head()

,duration,finished,path,source,target
0,62.250000,True,"[14th_century, 15th_century, 16th_century, Pac...",14th_century,African_slave_trade
1,66.000000,True,"[14th_century, Europe, Africa, Atlantic_slave_...",14th_century,African_slave_trade
2,59.142857,True,"[14th_century, Niger, Nigeria, British_Empire,...",14th_century,African_slave_trade
3,87.500000,True,"[14th_century, Italy, Roman_Catholic_Church, H...",14th_century,John_F._Kennedy
4,66.000000,True,"[14th_century, Europe, North_America, United_S...",14th_century,John_F._Kennedy


In [14]:
games_longer_than_4['first_click'] = games_longer_than_4['path'].apply(lambda x: x[1])
games_longer_than_4['second_click'] = games_longer_than_4['path'].apply(lambda x: x[2])
games_longer_than_4['third_click'] = games_longer_than_4['path'].apply(lambda x: x[3])
games_longer_than_4['num_back'] = games_longer_than_4.apply(lambda a: int(a['first_click'] == '<') + int(a['second_click'] == '<') + int(a['third_click'] == '<'), axis = 1)


#games_longer_than_4.drop(columns=['path'], inplace=True)

games_longer_than_4.head()


,duration,finished,path,source,target,first_click,second_click,third_click,num_back
0,62.250000,True,"[14th_century, 15th_century, 16th_century, Pac...",14th_century,African_slave_trade,15th_century,16th_century,Pacific_Ocean,0
1,66.000000,True,"[14th_century, Europe, Africa, Atlantic_slave_...",14th_century,African_slave_trade,Europe,Africa,Atlantic_slave_trade,0
2,59.142857,True,"[14th_century, Niger, Nigeria, British_Empire,...",14th_century,African_slave_trade,Niger,Nigeria,British_Empire,0
3,87.500000,True,"[14th_century, Italy, Roman_Catholic_Church, H...",14th_century,John_F._Kennedy,Italy,Roman_Catholic_Church,HIV,0
4,66.000000,True,"[14th_century, Europe, North_America, United_S...",14th_century,John_F._Kennedy,Europe,North_America,United_States,0


In [15]:
print((games_longer_than_4['first_click'] == '<').sum())
print((games_longer_than_4['second_click'] == '<').sum())
print((games_longer_than_4['third_click'] == '<').sum())
games_longer_than_4[games_longer_than_4['num_back'] > 0].shape



0
1235
1303


(2538, 9)

In [16]:
games_longer_than_4['second_is_<'] = games_longer_than_4['second_click'] == '<'
games_longer_than_4['third_is_<'] = games_longer_than_4['third_click'] == '<'

In [17]:
games_longer_than_4['second_click'] = games_longer_than_4.apply(
    lambda row: row['source'] if row['second_click'] == '<' else row['second_click'], axis=1
)

games_longer_than_4['third_click'] = games_longer_than_4.apply(
    lambda row: row['first_click'] if row['third_click'] == '<' else row['third_click'], axis=1
)
print((games_longer_than_4['first_click'] == '<').sum())
print((games_longer_than_4['second_click'] == '<').sum())
print((games_longer_than_4['third_click'] == '<').sum())

0
0
0


In [18]:
node_stats_df = load_or_compute_node_stats()
games_longer_than_4 = merge_with_node_data(games_longer_than_4, node_stats_df, columns = ['source', 'target', 'first_click', 'second_click', 'third_click'], data = ['pagerank'])
games_longer_than_4.head()

Loaded 4604 node stats


,duration,finished,path,source,target,first_click,second_click,third_click,num_back,second_is_<,third_is_<,pagerank_source,pagerank_target,pagerank_first_click,pagerank_second_click,pagerank_third_click
0,62.250000,True,"[14th_century, 15th_century, 16th_century, Pac...",14th_century,African_slave_trade,15th_century,16th_century,Pacific_Ocean,0,False,False,0.000642,0.000030,0.001024,0.001223,0.001408
1,66.000000,True,"[14th_century, Europe, Africa, Atlantic_slave_...",14th_century,African_slave_trade,Europe,Africa,Atlantic_slave_trade,0,False,False,0.000642,0.000030,0.006698,0.003321,0.000145
2,59.142857,True,"[14th_century, Niger, Nigeria, British_Empire,...",14th_century,African_slave_trade,Niger,Nigeria,British_Empire,0,False,False,0.000642,0.000030,0.000408,0.000630,0.001197
3,87.500000,True,"[14th_century, Italy, Roman_Catholic_Church, H...",14th_century,John_F._Kennedy,Italy,Roman_Catholic_Church,HIV,0,False,False,0.000642,0.000315,0.003975,0.002173,0.000357
4,66.000000,True,"[14th_century, Europe, North_America, United_S...",14th_century,John_F._Kennedy,Europe,North_America,United_States,0,False,False,0.000642,0.000315,0.006698,0.002751,0.009774


In [19]:
fame_df = load_fame()
games_longer_than_4 = merge_with_fame_data(games_longer_than_4, fame_df, columns = ['source', 'first_click', 'second_click', 'third_click'])

In [20]:
s = fame_df.isna()
s[s['fame_score'] == True]

,fame_score
article_name,


In [21]:
games_longer_than_4.head()

,duration,finished,path,source,target,first_click,second_click,third_click,num_back,second_is_<,third_is_<,pagerank_source,pagerank_target,pagerank_first_click,pagerank_second_click,pagerank_third_click,fame_score_source,fame_score_first_click,fame_score_second_click,fame_score_third_click
0,62.250000,True,"[14th_century, 15th_century, 16th_century, Pac...",14th_century,African_slave_trade,15th_century,16th_century,Pacific_Ocean,0,False,False,0.000642,0.000030,0.001024,0.001223,0.001408,8.0,8.0,8.0,9.0
1,66.000000,True,"[14th_century, Europe, Africa, Atlantic_slave_...",14th_century,African_slave_trade,Europe,Africa,Atlantic_slave_trade,0,False,False,0.000642,0.000030,0.006698,0.003321,0.000145,8.0,8.0,9.0,6.0
2,59.142857,True,"[14th_century, Niger, Nigeria, British_Empire,...",14th_century,African_slave_trade,Niger,Nigeria,British_Empire,0,False,False,0.000642,0.000030,0.000408,0.000630,0.001197,8.0,6.0,8.0,8.0
3,87.500000,True,"[14th_century, Italy, Roman_Catholic_Church, H...",14th_century,John_F._Kennedy,Italy,Roman_Catholic_Church,HIV,0,False,False,0.000642,0.000315,0.003975,0.002173,0.000357,8.0,9.0,9.0,9.0
4,66.000000,True,"[14th_century, Europe, North_America, United_S...",14th_century,John_F._Kennedy,Europe,North_America,United_States,0,False,False,0.000642,0.000315,0.006698,0.002751,0.009774,8.0,8.0,9.0,10.0


In [22]:
link_proba = load_link_proba()
link_proba.head()
links_df = load_links_df()
link_proba.isna().sum()
#print(print(games_longer_than_4.loc[3167]))
#links[(links['source'] == 'Finland') & (links['target'] == 'Åland')]

Loaded 119882 links in df of shape (119882, 2)


link_probability    0
dtype: int64

In [23]:
combs = []
seen_tuples = set() 

for i, row in games_longer_than_4.iterrows():
    pair1 = (row['source'], row['first_click'])
    if pair1 not in link_proba.index and pair1 not in seen_tuples:
        combs.append((i, *pair1))
        seen_tuples.add(pair1)

    pair2 = (row['first_click'], row['second_click'])
    sec_is_b = row['second_is_<']
    if pair2 not in link_proba.index and not sec_is_b and pair2 not in seen_tuples:
        combs.append((i, *pair2))
        seen_tuples.add(pair2)
    
    pair3 = (row['second_click'], row['third_click'])
    third_is_b = row['third_is_<']
    if pair3 not in link_proba.index and not third_is_b and pair3 not in seen_tuples:
        combs.append((i, *pair3))
        seen_tuples.add(pair3)
indexes = []
for i in combs:
    print(i)
    indexes.append(i[0])
print(indexes)

(935, 'Batman', 'Wikipedia_Text_of_the_GNU_Free_Documentation_License')
(3167, 'Finland', 'Åland')
(8055, 'Programming_language', 'Wikipedia_Text_of_the_GNU_Free_Documentation_License')
(15331, 'Consolation_of_Philosophy', 'Wikipedia_Text_of_the_GNU_Free_Documentation_License')
(15730, 'Abbasid', 'Wikipedia_Text_of_the_GNU_Free_Documentation_License')
(22757, 'Yttrium', 'Wikipedia_Text_of_the_GNU_Free_Documentation_License')
(28130, 'Company_(law)', 'Wikipedia_Text_of_the_GNU_Free_Documentation_License')
(32236, 'Rabbit', 'Wikipedia_Text_of_the_GNU_Free_Documentation_License')
(32412, 'Aircraft', 'Wikipedia_Text_of_the_GNU_Free_Documentation_License')
(34184, 'Communication', 'Wikipedia_Text_of_the_GNU_Free_Documentation_License')
(38446, 'Railway_post_office', 'Wikipedia_Text_of_the_GNU_Free_Documentation_License')
(42069, 'Electron_beam_welding', 'Wikipedia_Text_of_the_GNU_Free_Documentation_License')
(46398, 'Actinium', 'Wikipedia_Text_of_the_GNU_Free_Documentation_License')
[935, 3

In [24]:
games_longer_than_4.drop(index = indexes, inplace = True)

In [25]:
games_longer_than_4['second_click'] = games_longer_than_4.apply(lambda row: '<' if row['second_is_<'] else row['second_click'], axis = 1)
games_longer_than_4['third_click'] = games_longer_than_4.apply(lambda row: '<' if row['third_is_<'] else row['third_click'], axis = 1)

In [26]:
print((games_longer_than_4['first_click'] == '<').sum())
print((games_longer_than_4['second_click'] == '<').sum())
print((games_longer_than_4['third_click'] == '<').sum())

0
1226
1302


In [27]:
games_longer_than_4 = add_link_proba_info(games_longer_than_4, link_proba, pairs = [['source', 'first_click'], ['first_click', 'second_click'], ['second_click', 'third_click']], names = ['first_link', 'second_link', 'third_link'])
games_longer_than_4.head()

Dropped 2 games without link statistics


,duration,finished,path,source,target,first_click,second_click,third_click,num_back,second_is_<,...,pagerank_first_click,pagerank_second_click,pagerank_third_click,fame_score_source,fame_score_first_click,fame_score_second_click,fame_score_third_click,first_link,second_link,third_link
0,62.250000,True,"[14th_century, 15th_century, 16th_century, Pac...",14th_century,African_slave_trade,15th_century,16th_century,Pacific_Ocean,0,False,...,0.001024,0.001223,0.001408,8.0,8.0,8.0,9.0,0.9,0.60,0.2
1,66.000000,True,"[14th_century, Europe, Africa, Atlantic_slave_...",14th_century,African_slave_trade,Europe,Africa,Atlantic_slave_trade,0,False,...,0.006698,0.003321,0.000145,8.0,8.0,9.0,6.0,0.9,0.30,0.7
2,59.142857,True,"[14th_century, Niger, Nigeria, British_Empire,...",14th_century,African_slave_trade,Niger,Nigeria,British_Empire,0,False,...,0.000408,0.000630,0.001197,8.0,6.0,8.0,8.0,0.1,0.90,0.3
3,87.500000,True,"[14th_century, Italy, Roman_Catholic_Church, H...",14th_century,John_F._Kennedy,Italy,Roman_Catholic_Church,HIV,0,False,...,0.003975,0.002173,0.000357,8.0,9.0,9.0,9.0,0.4,0.25,0.2
4,66.000000,True,"[14th_century, Europe, North_America, United_S...",14th_century,John_F._Kennedy,Europe,North_America,United_States,0,False,...,0.006698,0.002751,0.009774,8.0,8.0,9.0,10.0,0.9,0.10,0.9


In [28]:
print((games_longer_than_4['first_click'] == '<').sum())
print((games_longer_than_4['second_click'] == '<').sum())
print((games_longer_than_4['third_click'] == '<').sum())

0
1224
1302


In [29]:
articles_df = load_article_df()
adj_matrix = construct_adjecency_matrix(links_df, articles_df['article_name'].tolist())
adj_list = from_adjacency_matrix_to_list(adj_matrix)
index_mapping = generate_inverse_index_mapping(adj_list)
pair_data = load_pair_data_with_multiindex('../src/data/pair_stats.txt', index_mapping)
pair_data.head()

Loaded 4604 articles in df of shape (4604, 1)


shortest_path_length  shortest_path_count  \
source       target                                                    
10th_century 10th_century                     0                    1   
             11th_century                     1                    1   
             12th_century                     2                    5   
             13th_century                     2                    4   
             14th_century                     2                    4   

                           max_sp_node_degree  max_sp_avg_node_degree  \
source       target                                                     
10th_century 10th_century                   0                       0   
             11th_century                  26                      13   
             12th_century                  48                      24   
             13th_century                  79                      35   
             14th_century                  53                      26   

                           avg_sp_avg_node_degree  one_longer_path_count  \
source       target                                                        
10th_century 10th_century                       0                      0   
             11th_century                      13                      2   
             12th_century                      23                    121   
             13th_century                      28                    131   
             14th_century                      22                    113   

                           max_ol_node_degree  max_ol_avg_node_degree  \
source       target                                                     
10th_century 10th_century                   0                       0   
             11th_century                  62                      29   
             12th_century                 180                      93   
             13th_century                 169                      84   
             14th_century                 169                      84   

                           avg_ol_avg_node_degree  two_longer_path_count  \
source       target                                                        
10th_century 10th_century                       0                      0   
             11th_century                      26                      7   
             12th_century                      37                     30   
             13th_century                      39                     30   
             14th_century                      37                     30   

                           max_tl_node_degree  max_tl_avg_node_degree  
source       target                                                    
10th_century 10th_century                   0                       0  
             11th_century                 112                      46  
             12th_century                 169                      70  
             13th_century                 169                      71  
             14th_century                 169                      70

In [30]:
pair_data.columns

Index(['shortest_path_length', 'shortest_path_count', 'max_sp_node_degree',
       'max_sp_avg_node_degree', 'avg_sp_avg_node_degree',
       'one_longer_path_count', 'max_ol_node_degree', 'max_ol_avg_node_degree',
       'avg_ol_avg_node_degree', 'two_longer_path_count', 'max_tl_node_degree',
       'max_tl_avg_node_degree'],
      dtype='object')

In [31]:
games_longer_than_4 = add_pair_data(games_longer_than_4, pair_data, pairs =[['source', 'target'], ['first_click', 'target'], ['second_click', 'target'], ['third_click', 'target']], names = ["source", "first", "second", "third"], data = ['shortest_path_length', 'shortest_path_count']) 

Dropped 3 games without link statistics


In [32]:
games_longer_than_4.head()

,duration,finished,path,source,target,first_click,second_click,third_click,num_back,second_is_<,...,second_link,third_link,shortest_path_length_source,shortest_path_count_source,shortest_path_length_first,shortest_path_count_first,shortest_path_length_second,shortest_path_count_second,shortest_path_length_third,shortest_path_count_third
0,62.250000,True,"[14th_century, 15th_century, 16th_century, Pac...",14th_century,African_slave_trade,15th_century,16th_century,Pacific_Ocean,0,False,...,0.60,0.2,3.0,3.0,3.0,3.0,3.0,6.0,3.0,3.0
1,66.000000,True,"[14th_century, Europe, Africa, Atlantic_slave_...",14th_century,African_slave_trade,Europe,Africa,Atlantic_slave_trade,0,False,...,0.30,0.7,3.0,3.0,3.0,7.0,2.0,1.0,1.0,1.0
2,59.142857,True,"[14th_century, Niger, Nigeria, British_Empire,...",14th_century,African_slave_trade,Niger,Nigeria,British_Empire,0,False,...,0.90,0.3,3.0,3.0,3.0,4.0,2.0,1.0,3.0,7.0
3,87.500000,True,"[14th_century, Italy, Roman_Catholic_Church, H...",14th_century,John_F._Kennedy,Italy,Roman_Catholic_Church,HIV,0,False,...,0.25,0.2,3.0,18.0,2.0,1.0,2.0,1.0,2.0,2.0
4,66.000000,True,"[14th_century, Europe, North_America, United_S...",14th_century,John_F._Kennedy,Europe,North_America,United_States,0,False,...,0.10,0.9,3.0,18.0,2.0,2.0,2.0,2.0,1.0,1.0


In [33]:
games_longer_than_4.columns

Index(['duration', 'finished', 'path', 'source', 'target', 'first_click',
       'second_click', 'third_click', 'num_back', 'second_is_<', 'third_is_<',
       'pagerank_source', 'pagerank_target', 'pagerank_first_click',
       'pagerank_second_click', 'pagerank_third_click', 'fame_score_source',
       'fame_score_first_click', 'fame_score_second_click',
       'fame_score_third_click', 'first_link', 'second_link', 'third_link',
       'shortest_path_length_source', 'shortest_path_count_source',
       'shortest_path_length_first', 'shortest_path_count_first',
       'shortest_path_length_second', 'shortest_path_count_second',
       'shortest_path_length_third', 'shortest_path_count_third'],
      dtype='object')

In [34]:
games_longer_than_4.drop(columns = ['path', 'source', 'target', 'first_click',
       'second_click', 'third_click', 'num_back'], inplace = True)

In [35]:
games_longer_than_4.head()

,duration,finished,second_is_<,third_is_<,pagerank_source,pagerank_target,pagerank_first_click,pagerank_second_click,pagerank_third_click,fame_score_source,...,second_link,third_link,shortest_path_length_source,shortest_path_count_source,shortest_path_length_first,shortest_path_count_first,shortest_path_length_second,shortest_path_count_second,shortest_path_length_third,shortest_path_count_third
0,62.250000,True,False,False,0.000642,0.000030,0.001024,0.001223,0.001408,8.0,...,0.60,0.2,3.0,3.0,3.0,3.0,3.0,6.0,3.0,3.0
1,66.000000,True,False,False,0.000642,0.000030,0.006698,0.003321,0.000145,8.0,...,0.30,0.7,3.0,3.0,3.0,7.0,2.0,1.0,1.0,1.0
2,59.142857,True,False,False,0.000642,0.000030,0.000408,0.000630,0.001197,8.0,...,0.90,0.3,3.0,3.0,3.0,4.0,2.0,1.0,3.0,7.0
3,87.500000,True,False,False,0.000642,0.000315,0.003975,0.002173,0.000357,8.0,...,0.25,0.2,3.0,18.0,2.0,1.0,2.0,1.0,2.0,2.0
4,66.000000,True,False,False,0.000642,0.000315,0.006698,0.002751,0.009774,8.0,...,0.10,0.9,3.0,18.0,2.0,2.0,2.0,2.0,1.0,1.0


In [36]:
games_longer_than_4.columns

Index(['duration', 'finished', 'second_is_<', 'third_is_<', 'pagerank_source',
       'pagerank_target', 'pagerank_first_click', 'pagerank_second_click',
       'pagerank_third_click', 'fame_score_source', 'fame_score_first_click',
       'fame_score_second_click', 'fame_score_third_click', 'first_link',
       'second_link', 'third_link', 'shortest_path_length_source',
       'shortest_path_count_source', 'shortest_path_length_first',
       'shortest_path_count_first', 'shortest_path_length_second',
       'shortest_path_count_second', 'shortest_path_length_third',
       'shortest_path_count_third'],
      dtype='object')

In [37]:
features_1 = ['duration', 'second_is_<', 'third_is_<', 'pagerank_source',
       'pagerank_target', 'pagerank_first_click', 'pagerank_second_click',
       'pagerank_third_click', 'fame_score_source', 'fame_score_first_click',
       'fame_score_second_click', 'fame_score_third_click', 'first_link',
       'second_link', 'third_link', 'shortest_path_length_source',
       'shortest_path_count_source', 'shortest_path_length_first',
       'shortest_path_count_first', 'shortest_path_length_second',
       'shortest_path_count_second', 'shortest_path_length_third',
       'shortest_path_count_third']

In [38]:
model_1 = LogisticRegression(games_longer_than_4, features_1)
model_1.fit()

Class distribution: finished
False    0.5
True     0.5
Name: proportion, dtype: float64
Total number of samples: 13694
Optimization terminated successfully.
         Current function value: 0.534618
         Iterations 6
Training Set Metrics:
Threshold:   0.5790
F1 Score:    0.7325
Precision:   0.7373
Accuracy:    0.7335
              precision    recall  f1-score   support

       False     0.7016    0.7964    0.7460      1370
        True     0.7644    0.6611    0.7090      1369

    accuracy                         0.7287      2739
   macro avg     0.7330    0.7287    0.7275      2739
weighted avg     0.7330    0.7287    0.7275      2739



In [39]:
model_1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:               finished   No. Observations:                10955
Model:                          Logit   Df Residuals:                    10932
Method:                           MLE   Df Model:                           22
Date:                Thu, 12 Dec 2024   Pseudo R-squ.:                  0.2287
Time:                        17:49:51   Log-Likelihood:                -5856.7
converged:                       True   LL-Null:                       -7593.4
Covariance Type:            nonrobust   LLR p-value:                     0.000
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
x1            -0.5400      0.040    -13.646      0.000      -0.618      -0.462
x2            -0.0358      0.040     -0.890      0.374      -0.115       0.043
x3            -0.8761      0.033    -26.644      0.000      -0.941      -0.812
x4            -0.0043      0.025     -0.173      0.862      -0.052       0.044
x5             0.2732      0.038      7.108      0.000       0.198       0.349
x6             0.0060      0.026      0.232      0.817      -0.045       0.057
x7            -0.0521      0.025     -2.043      0.041      -0.102      -0.002
x8            -0.1113      0.025     -4.541      0.000      -0.159      -0.063
x9             0.0307      0.024      1.276      0.202      -0.016       0.078
x10           -0.0275      0.025     -1.115      0.265      -0.076       0.021
x11            0.0230      0.025      0.925      0.355      -0.026       0.072
x12           -0.0291      0.025     -1.179      0.238      -0.078       0.019
x13            0.0052      0.023      0.225      0.822      -0.040       0.050
x14           -0.0023      0.024     -0.093      0.926      -0.050       0.045
x15           -0.0003      0.026     -0.010      0.992      -0.052       0.052
x16            0.0662      0.039      1.693      0.090      -0.010       0.143
x17           -0.0423      0.027     -1.582      0.114      -0.095       0.010
x18            0.0505      0.042      1.215      0.224      -0.031       0.132
x19            0.0558      0.027      2.072      0.038       0.003       0.109
x20           -0.1715      0.051     -3.339      0.001      -0.272      -0.071
x21            0.0953      0.027      3.589      0.000       0.043       0.147
x22           -1.4804      0.046    -31.925      0.000      -1.571      -1.389
x23            0.1958      0.026      7.608      0.000       0.145       0.246
==============================================================================
"""

Based on p-value, we remove 'second_is_<', 'pagerank_source', 'pagerank_first_click', 'fame_score_source', 'fame_score_first_click', 'fame_score_second_click', 'fame_score_third_click', 'first_link', 'second_link', 'third_link', 'shortest_path_count_source', 'shortest_path_length_first'.

In [77]:
features_2 = ['duration', 'third_is_<',
       'pagerank_target', 
       'pagerank_third_click',
       'pagerank_second_click',  'shortest_path_length_source',
        'shortest_path_length_second',
       'shortest_path_count_second', 'shortest_path_length_third',
       'shortest_path_count_third']

In [78]:
model_2 = LogisticRegression(games_longer_than_4, features_2)
model_2.fit()

Class distribution: finished
False    0.5
True     0.5
Name: proportion, dtype: float64
Total number of samples: 13694
Optimization terminated successfully.
         Current function value: 0.535540
         Iterations 6
Training Set Metrics:
Threshold:   0.5840
F1 Score:    0.7318
Precision:   0.7371
Accuracy:    0.7330
              precision    recall  f1-score   support

       False     0.7016    0.8066    0.7504      1370
        True     0.7723    0.6567    0.7098      1369

    accuracy                         0.7317      2739
   macro avg     0.7370    0.7316    0.7301      2739
weighted avg     0.7369    0.7317    0.7301      2739



In [79]:
model_2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:               finished   No. Observations:                10955
Model:                          Logit   Df Residuals:                    10945
Method:                           MLE   Df Model:                            9
Date:                Thu, 12 Dec 2024   Pseudo R-squ.:                  0.2274
Time:                        18:08:38   Log-Likelihood:                -5866.8
converged:                       True   LL-Null:                       -7593.4
Covariance Type:            nonrobust   LLR p-value:                     0.000
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
x1            -0.5405      0.039    -13.741      0.000      -0.618      -0.463
x2            -0.8781      0.029    -29.777      0.000      -0.936      -0.820
x3             0.2661      0.036      7.316      0.000       0.195       0.337
x4            -0.1239      0.023     -5.502      0.000      -0.168      -0.080
x5            -0.0406      0.023     -1.759      0.079      -0.086       0.005
x6             0.0550      0.029      1.879      0.060      -0.002       0.112
x7            -0.1194      0.029     -4.068      0.000      -0.177      -0.062
x8             0.0866      0.025      3.512      0.000       0.038       0.135
x9            -1.4752      0.042    -35.221      0.000      -1.557      -1.393
x10            0.1978      0.025      7.796      0.000       0.148       0.248
==============================================================================
"""

Let's see if using an average of node values helps.

In [43]:
features_3 = ['duration', 'second_is_<', 'third_is_<',
       'pagerank_target', 'pagerank_second_click',
       'pagerank_third_click',
        'shortest_path_length_source',
        'shortest_path_length_second',
       'shortest_path_count_second', 'shortest_path_length_third',
       'shortest_path_count_third']

In [44]:
model_3 = LogisticRegression(games_longer_than_4, features_3)
model_3.fit()

Class distribution: finished
False    0.5
True     0.5
Name: proportion, dtype: float64
Total number of samples: 13694
Optimization terminated successfully.
         Current function value: 0.535536
         Iterations 6
Training Set Metrics:
Threshold:   0.5830
F1 Score:    0.7321
Precision:   0.7371
Accuracy:    0.7332
              precision    recall  f1-score   support

       False     0.7021    0.8051    0.7501      1370
        True     0.7714    0.6581    0.7103      1369

    accuracy                         0.7317      2739
   macro avg     0.7368    0.7316    0.7302      2739
weighted avg     0.7367    0.7317    0.7302      2739

